## Dataset --> [kaggle](https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign)

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from sklearn.metrics import accuracy_score
from keras.models import load_model
from tensorflow import argmax

# Data preprocessing

In [ ]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()

#Retrieving the images and their labels
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)

    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

In [ ]:
data = np.array(data)
labels = np.array(labels)

data.shape, labels.shape

In [ ]:
#Splitting training and validation dataset
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

# Model training

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]),
    Conv2D(filters=32, kernel_size=(5,5), activation='relu'),

    MaxPool2D(pool_size=(2, 2)),
    Dropout(rate=0.25),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),

    MaxPool2D(pool_size=(2, 2)),
    Dropout(rate=0.25),

    Flatten(),

    Dense(256, activation='relu'),
    Dropout(rate=0.5),

    Dense(43, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=15, validation_data=(X_val, y_val))

In [ ]:
model.save("traffic_classifier.h5")

In [2]:
model = load_model('traffic_classifier.h5')

In [19]:
# # visualizing loss functions
# model_losses = pd.DataFrame(model.history)
# model_losses.plot()

# Model Testing

In [4]:
y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values

In [5]:
data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))

X_test=np.array(data)

In [8]:
pred = model.predict(X_test)

395/395 [==============================] - 7s 19ms/step


In [41]:
predicted_classes = argmax(pred, axis=1)

In [50]:
#dictionary to label all traffic signs class.
classes = { 1:'Speed limit (20km/h)',
            2:'Speed limit (30km/h)',      
            3:'Speed limit (50km/h)',       
            4:'Speed limit (60km/h)',      
            5:'Speed limit (70km/h)',    
            6:'Speed limit (80km/h)',      
            7:'End of speed limit (80km/h)',     
            8:'Speed limit (100km/h)',    
            9:'Speed limit (120km/h)',     
           10:'No passing',   
           11:'No passing veh over 3.5 tons',     
           12:'Right-of-way at intersection',     
           13:'Priority road',    
           14:'Yield',     
           15:'Stop',       
           16:'No vehicles',       
           17:'Veh > 3.5 tons prohibited',       
           18:'No entry',       
           19:'General caution',     
           20:'Dangerous curve left',      
           21:'Dangerous curve right',   
           22:'Double curve',      
           23:'Bumpy road',     
           24:'Slippery road',       
           25:'Road narrows on the right',  
           26:'Road work',    
           27:'Traffic signals',      
           28:'Pedestrians',     
           29:'Children crossing',     
           30:'Bicycles crossing',       
           31:'Beware of ice/snow',
           32:'Wild animals crossing',      
           33:'End speed + passing limits',      
           34:'Turn right ahead',     
           35:'Turn left ahead',       
           36:'Ahead only',      
           37:'Go straight or right',      
           38:'Go straight or left',      
           39:'Keep right',     
           40:'Keep left',      
           41:'Roundabout mandatory',     
           42:'End of no passing',      
           43:'End no passing veh > 3.5 tons' }

In [62]:
predicted_classes, labels

(<tf.Tensor: shape=(12630,), dtype=int64, numpy=array([16,  1, 38, ...,  6,  7, 10], dtype=int64)>,
 array([16,  1, 38, ...,  6,  7, 10], dtype=int64))

In [74]:
for i in range(10):
    # Generate a random integer between 0 and len(X_test)-1
    random_integer = tf.cast(tf.random.uniform(shape=(), minval=0, maxval=len(X_test)), tf.int32)
    print('Actual - ' + classes[labels[random_integer]] + ', Predicted - ' + classes[int(predicted_classes[random_integer])])

Actual - Go straight or left, Predicted - Go straight or left
Actual - Yield, Predicted - Yield
Actual - Ahead only, Predicted - Ahead only
Actual - No passing, Predicted - No passing
Actual - Road narrows on the right, Predicted - Road narrows on the right
Actual - End of speed limit (80km/h), Predicted - End of speed limit (80km/h)
Actual - Priority road, Predicted - Priority road
Actual - No passing, Predicted - No passing
Actual - No passing veh over 3.5 tons, Predicted - No passing veh over 3.5 tons
Actual - Go straight or left, Predicted - Go straight or left
